In [20]:
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA

# Step 1: Data Preprocessing
data = pd.read_csv('train.csv')
data1 = pd.read_csv('test.csv')
data.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)
data1.drop(columns=['ID','Candidate','Constituency ∇'], inplace=True)

def convert_to_lakhs(value):
    value = str(value)
    if value == "0":
        return 0
    elif 'Crore+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number * 100)
    elif 'Lac+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number)
    elif 'Thou+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 100)
    elif 'Hund+' in value:
        number, unit = value.split()
        number = float(number.replace(",", ""))
        return int(number / 1000)
    else:
        return int(value)

data["Total Assets"] = data["Total Assets"].apply(convert_to_lakhs)
data["Liabilities"] = data["Liabilities"].apply(convert_to_lakhs)
data1["Total Assets"] = data1["Total Assets"].apply(convert_to_lakhs)
data1["Liabilities"] = data1["Liabilities"].apply(convert_to_lakhs)

# Perform label encoding for Education column

from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
data['Education'] = label_encoder.fit_transform(data['Education'])

# data['Education'] = data['Education'].astype('category').cat.codes

data = pd.get_dummies(data, columns=['Party','state'])
data1 = pd.get_dummies(data1, columns=['Party','state'])
data

,Criminal Case,Total Assets,Liabilities,Education,Diff,Party_AAP,Party_AIADMK,Party_AITC,Party_BJD,Party_BJP,...,state_ODISHA,state_PUDUCHERRY,state_PUNJAB,state_RAJASTHAN,state_SIKKIM,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL
0,4,21100,200,3,20900,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,0,100,0,1,100,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,0,700,22,9,678,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,900,24,9,876,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,2,200,61,3,139,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,1,61,10,6,51,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2055,0,200,8,0,192,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
2056,0,1300,85,5,1215,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
2057,1,2500,94,1,2406,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import KFold

# Assuming 'data' is your original DataFrame
X = data.drop(columns=['Education'])
y = data['Education']

kf = KFold(n_splits=5, shuffle=True, random_state=42)
# ans = 0
# Iterate through the folds and train the Bernoulli Naive Bayes model
# for train_index, test_index in kf.split(data):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     # Train the Bernoulli Naive Bayes model
#     bernoulli_nb = BernoulliNB(
#         alpha=1,
#         binarize=0.1,
#         fit_prior=True,
#         class_prior=None
#     )
#     bernoulli_nb.fit(X_train, y_train)

#     # Evaluate the model on the test set
#     score = bernoulli_nb.score(X_test, y_test)
#     print(f"Accuracy on fold: {score:.10f}")
#     ans += score

# print(ans/5)

bernoulli_nb = BernoulliNB(
    alpha=1.0,
    binarize=0.0,
    fit_prior=True,
    class_prior=None
)
bernoulli_nb.fit(X, y)

predictions = bernoulli_nb.predict(data1)
predictions = label_encoder.inverse_transform(predictions)
df_pred = pd.DataFrame(predictions, columns=["Education"])
df_pred.to_csv('12.csv', index_label='ID')